In [1]:
import requests
import pandas as pd
import datetime

In [2]:
import xmltodict
import json

In [3]:
import re

## Daten herunterladen von API und umwandeln

In [5]:
#Abfrage für alle Vorstösse einer Partei im gewählten Zeitrahmen (1.1.14 bis 1.1.24)
r = requests.get('http://www.gemeinderat-zuerich.ch/api/geschaeft/searchdetails?q=partei all "GLP" AND beginn_start > "2014-01-01 00:00:00" AND beginn_start < "2024-01-01 00:00:00" sortBy beginn_start/sort.ascending&l=de-CH')

In [6]:
#Die Arbeit mit den XML-Daten funktioniert schlecht, darum umwandeln in JSON
xml_data = r.text
data_dict = xmltodict.parse(xml_data)

## DataFrame mit Vorstössen erstellen

In [7]:
# Minidict mit den wichtigsten Angaben zu den Vorstössen. Da es nicht immer einen Mitunterzeichner gibt, braucht's einen Fail-Safe
vorstoesse = []

for hit in data_dict["SearchDetailResponse"]["Hit"]:
    titel = hit["Geschaeft"]["Titel"]
    geschaeftsart = hit["Geschaeft"]["Geschaeftsart"]
    ID = hit["Geschaeft"]["@OBJ_GUID"]
    datum = hit["Geschaeft"]["Beginn"]["Start"]["#text"]
    erstunterzeichner = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Name"]
    partei_erst = hit["Geschaeft"]["Erstunterzeichner"]["KontaktGremium"]["Partei"]
    
    # Kontrollieren ob "Mitunterzeichner" existiert
    mitunterzeichner_data = hit["Geschaeft"].get("Mitunterzeichner")
    
    if mitunterzeichner_data:
        # Wenn es einen Mitunterzeichner gibt, dann Daten herunterladen
        mitunterzeichner = mitunterzeichner_data["KontaktGremium"]["Name"]
        partei_mit = mitunterzeichner_data["KontaktGremium"]["Partei"]
    else:
        # Wenn nicht, dann None einsetzen
        mitunterzeichner = None
        partei_mit = None
    
    status = hit["Geschaeft"]["Geschaeftsstatus"]
    
    mini_dict = {"Titel": titel,
                 "Geschäftsart": geschaeftsart,
                 "ID": ID,
                 "Datum": datum,
                 "Erstunterzeichner": erstunterzeichner,
                 "Partei Erstunterzeichner": partei_erst,
                 "Mitunterzeichner": mitunterzeichner,
                 "Partei Mitunterzeichner": partei_mit,
                 "Geschäftsstatus": status}
    
    vorstoesse.append(mini_dict)

In [8]:
df_partei = pd.DataFrame(vorstoesse)

In [9]:
df_partei

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus
0,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05T00:00:00.000,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen
1,Kommunale Wohnüberbauung auf dem Tramdepot Har...,Postulat,a5573cb80d054660a8565a27eab1a841,2014-02-26T00:00:00.000,Martin Luchsinger,GLP,Jean-Claude Virchaux,CVP,Abgeschlossen
2,Durchgehende Veloverbindung aus dem Lettenquar...,Postulat,94f2be0b2acd4eab8e6d7c743304b799,2014-03-05T00:00:00.000,Simone Brander,SP,Guido Trevisan,GLP,Abgeschlossen
3,Studie zur Wirkung der Mobilitätskampagne des ...,Schriftliche Anfrage,ef16480862ec4387991f9b736a9378ba,2014-03-12T00:00:00.000,Samuel Dubno,GLP,Walter Angst,AL,Abgeschlossen
4,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19T00:00:00.000,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen
...,...,...,...,...,...,...,...,...,...
321,"Versorgungsengpässe bei Arzneimitteln, betroff...",Schriftliche Anfrage,5ab193cebc1a48559449633b072e7c49,2023-11-22T00:00:00.000,Matthias Renggli,SP,Patrick Hässig,GLP,InBearbeitung
322,Einsatz von Doppelgelenk-Trolleybussen bei sch...,Schriftliche Anfrage,f80960ad45a74f1e80e5a1cf2e3c16d7,2023-12-06T00:00:00.000,Ann-Catherine Nabholz,GLP,Ivo Bieri,SP,InBearbeitung
323,Angespannte Situation betreffend Mischverkehr ...,Schriftliche Anfrage,b16e4670c0fd48eba971adbf33c9bbf2,2023-12-06T00:00:00.000,Selina Frey,GLP,Serap Kahriman,GLP,InBearbeitung
324,"Einführung der Tagesschulen in der Stadt, Erre...",Schriftliche Anfrage,f0ffddee814242d8ac6e093291be6187,2023-12-06T00:00:00.000,Florine Angele,GLP,Christine Huber,GLP,InBearbeitung


## Resultate für Motionen und Postulate herunterladen

In [10]:
#Hiermit hole ich mir die Resultate für die Motionen und Postulate. 
#Es braucht einen doppelten Loop und Problembehebungsmassnahmen

entscheide = []

for hit in data_dict['SearchDetailResponse']['Hit']:
    # Kontrollieren, ob Ablaufschritte existiert und eine Liste ist
    ablaufschritte = hit['Geschaeft'].get('Ablaufschritte', {}).get('Aufgabe', [])
    
    if not isinstance(ablaufschritte, list):
        # Falls Aufgabe keine Liste ist, wird sie hier in eine Liste umgewandelt
        ablaufschritte = [ablaufschritte]
    
    for aufgabe in ablaufschritte:
        # Kontrollieren, ob AblaufschrittName existiert
        entscheid = aufgabe.get('AblaufschrittName')
        if entscheid is not None:
            entscheide.append({
                "HitID": hit["Geschaeft"]["@OBJ_GUID"],
                "Entscheid": entscheid,
            })

df_entscheid = pd.DataFrame(entscheide)

In [11]:
df_entscheid

,HitID,Entscheid
0,1ae08ec9dfc0465ba1c5491de2584218,"Eingang, Frist 3 Monate"
1,1ae08ec9dfc0465ba1c5491de2584218,"Stadtrat, Entgegennahme"
2,1ae08ec9dfc0465ba1c5491de2584218,"Ablehnung, beantragt"
3,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate"
4,1ae08ec9dfc0465ba1c5491de2584218,"Abschreibung, Geschäftsbericht"
...,...,...
1235,b16e4670c0fd48eba971adbf33c9bbf2,"Eingang, Frist 3 Monate"
1236,f0ffddee814242d8ac6e093291be6187,"Eingang, Frist 3 Monate"
1237,46c4378b24f247519883348f7250e8a8,Eingang
1238,46c4378b24f247519883348f7250e8a8,"Stadtrat, Entgegennahme"


## Daten putzen

In [12]:
#Zu jedem Geschäft gibt es mehrere Verfahrensschritte. 
#Mich interessieren nur die Schlussentscheide, die ich mit den keywords heraushole

keywords = ["Überweisung", "Ablehnung", "Rückzug", "Umwandlung"]

# Ich kann ein Pattern kreieren mit meinen keyword, zusammengefügt werden sie mit dem OR operator (|)
pattern = '|'.join(map(re.escape, keywords))

# Jetzt kann ich das Pattern auf meine Entscheidspalte anwenden mit str.contains
df_entscheid_neu = df_entscheid[df_entscheid.Entscheid.str.contains(pattern, case=False, regex=True)]

In [13]:
df_entscheid_neu

,HitID,Entscheid
2,1ae08ec9dfc0465ba1c5491de2584218,"Ablehnung, beantragt"
3,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate"
6,a5573cb80d054660a8565a27eab1a841,"Überweisung, Frist 24 Monate"
11,94f2be0b2acd4eab8e6d7c743304b799,"Ablehnung, beantragt"
12,94f2be0b2acd4eab8e6d7c743304b799,"Überweisung, Frist 24 Monate"
...,...,...
1211,6f593aa35dd74433bd7052f883fb57a8,"Überweisung, Frist 24 Monate"
1214,d0cfc9e0e6ff4b45a3d9e498429947c3,"Ablehnung, beantragt"
1217,d0cfc9e0e6ff4b45a3d9e498429947c3,"Überweisung, Frist 24 Monate"
1220,08dc2ce950d44c86969958d80ea62002,"Ablehnung, beantragt"


In [14]:
# Das Problem ist noch nicht ganz gelöst, zwei Verfahrensschritte sind geblieben, die ich nicht brauche: 
exclude_values = ["Stadtrat, Ablehnung", "Ablehnung, beantragt"]

# Für das bereinigte DataFrame behalte ich nun alle Zeilen, welche die exclude-values nicht enthalten
df_entscheid_clean = df_entscheid_neu[~df_entscheid_neu['Entscheid'].isin(exclude_values)]

In [15]:
df_entscheid_clean

,HitID,Entscheid
3,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate"
6,a5573cb80d054660a8565a27eab1a841,"Überweisung, Frist 24 Monate"
12,94f2be0b2acd4eab8e6d7c743304b799,"Überweisung, Frist 24 Monate"
17,4940b23f724d4d6cb4a21aed8f043f63,"Überweisung, Frist 24 Monate"
20,dc9c3c794faf4fc2bbe95377f4a88d98,Umwandlung in Postulat
...,...,...
1183,ef35724a95744fe281b6fdbd42a9b869,"Überweisung, Frist 24 Monate"
1194,764727820c4a472d9ddeeaa42e989043,"Überweisung, Frist 24 Monate"
1211,6f593aa35dd74433bd7052f883fb57a8,"Überweisung, Frist 24 Monate"
1217,d0cfc9e0e6ff4b45a3d9e498429947c3,"Überweisung, Frist 24 Monate"


In [16]:
#Nun sind noch verschiedene Varianten für Überweisung verblieben, die ich noch vereinheitliche und in neuer Spalte speichere
def clean(value):
    if 'Überweisung' in value:
        return 'überweisung'
    
    return value

#df_entscheid_clean["_entscheidbereinigt"] = df_entscheid_clean["Entscheid"].apply(clean)
df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']
df_entscheid_clean.loc[df_entscheid_clean['_entscheidbereinigt'].str.contains("Überweisung"), '_entscheidbereinigt'] = "Überweisung"
df_entscheid_clean

C:\Users\hudecj\AppData\Local\Temp\ipykernel_77780\3655237080.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entscheid_clean['_entscheidbereinigt'] = df_entscheid_clean['Entscheid']


,HitID,Entscheid,_entscheidbereinigt
3,1ae08ec9dfc0465ba1c5491de2584218,"Überweisung, Frist 24 Monate",Überweisung
6,a5573cb80d054660a8565a27eab1a841,"Überweisung, Frist 24 Monate",Überweisung
12,94f2be0b2acd4eab8e6d7c743304b799,"Überweisung, Frist 24 Monate",Überweisung
17,4940b23f724d4d6cb4a21aed8f043f63,"Überweisung, Frist 24 Monate",Überweisung
20,dc9c3c794faf4fc2bbe95377f4a88d98,Umwandlung in Postulat,Umwandlung in Postulat
...,...,...,...
1183,ef35724a95744fe281b6fdbd42a9b869,"Überweisung, Frist 24 Monate",Überweisung
1194,764727820c4a472d9ddeeaa42e989043,"Überweisung, Frist 24 Monate",Überweisung
1211,6f593aa35dd74433bd7052f883fb57a8,"Überweisung, Frist 24 Monate",Überweisung
1217,d0cfc9e0e6ff4b45a3d9e498429947c3,"Überweisung, Frist 24 Monate",Überweisung


In [17]:
#Hier führe ich die beiden Tabellen zusammen anhand der ID resp. HitID

df_neu = df_partei.copy()

df_partei_final = df_neu.merge(df_entscheid_clean, how='left', left_on='ID', right_on='HitID')[["Titel", "Geschäftsart", "ID", "Datum", "Erstunterzeichner", "Partei Erstunterzeichner", "Mitunterzeichner", "Partei Mitunterzeichner", "Geschäftsstatus", "Entscheid", '_entscheidbereinigt']]
df_partei_final

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05T00:00:00.000,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
1,Kommunale Wohnüberbauung auf dem Tramdepot Har...,Postulat,a5573cb80d054660a8565a27eab1a841,2014-02-26T00:00:00.000,Martin Luchsinger,GLP,Jean-Claude Virchaux,CVP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
2,Durchgehende Veloverbindung aus dem Lettenquar...,Postulat,94f2be0b2acd4eab8e6d7c743304b799,2014-03-05T00:00:00.000,Simone Brander,SP,Guido Trevisan,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
3,Studie zur Wirkung der Mobilitätskampagne des ...,Schriftliche Anfrage,ef16480862ec4387991f9b736a9378ba,2014-03-12T00:00:00.000,Samuel Dubno,GLP,Walter Angst,AL,Abgeschlossen,NaN,NaN
4,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19T00:00:00.000,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
...,...,...,...,...,...,...,...,...,...,...,...
324,"Versorgungsengpässe bei Arzneimitteln, betroff...",Schriftliche Anfrage,5ab193cebc1a48559449633b072e7c49,2023-11-22T00:00:00.000,Matthias Renggli,SP,Patrick Hässig,GLP,InBearbeitung,NaN,NaN
325,Einsatz von Doppelgelenk-Trolleybussen bei sch...,Schriftliche Anfrage,f80960ad45a74f1e80e5a1cf2e3c16d7,2023-12-06T00:00:00.000,Ann-Catherine Nabholz,GLP,Ivo Bieri,SP,InBearbeitung,NaN,NaN
326,Angespannte Situation betreffend Mischverkehr ...,Schriftliche Anfrage,b16e4670c0fd48eba971adbf33c9bbf2,2023-12-06T00:00:00.000,Selina Frey,GLP,Serap Kahriman,GLP,InBearbeitung,NaN,NaN
327,"Einführung der Tagesschulen in der Stadt, Erre...",Schriftliche Anfrage,f0ffddee814242d8ac6e093291be6187,2023-12-06T00:00:00.000,Florine Angele,GLP,Christine Huber,GLP,InBearbeitung,NaN,NaN


In [18]:
#Diese Resultate speichern
df_partei_final.to_csv('GLP_Vorstoesse_14_23_mit_Resultat.csv', index=False)

## Das Problem mit den umgewandelten Postulaten lösen

In [19]:
#Ein Problem bleibt noch: Bei Motionen, die in Postulate umgewandelt wurden, ist das Ergebnis nicht verzeichnet. 
#Weil es wenige Fälle sind, schaue ich das von Hand in den dazugehörigen PDF nach.
df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
5,"Anpassung der Datenschutzverordnung, Streichun...",Motion,dc9c3c794faf4fc2bbe95377f4a88d98,2014-04-02T00:00:00.000,Samuel Dubno,GLP,Ann-Catherine Nabholz,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
96,Neue Nutzung für den bisherigen Schiessstand H...,Motion,b3b7b404584d4a0b9232f4e9f1a3bd5f,2017-12-06T00:00:00.000,Pascal Lamprecht,SP,Markus Baumann,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
102,Einführung einer einheitlichen digitalen ID fü...,Motion,ade2db27c44e40c0927562f5aaa2292a,2018-01-31T00:00:00.000,Isabel Garcia,FDP,Corina Gredig,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
119,Aufhebung des Schwimmverbots in der Limmat auf...,Motion,b29d97daf827426eb59a265c7c7608b8,2018-07-11T00:00:00.000,Guido Hüni,GLP,Shaibal Roy,GLP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
170,Zulassung von Ausländerinnen und Ausländern mi...,Motion,3e4d684d690549d0b8e4e4f5adbc47b3,2019-08-21T00:00:00.000,Shaibal Roy,GLP,Marcel Bührig,Grüne,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
173,Vollständige Deckung des Gasbedarfs der Stadt ...,Motion,559cba49e3c24a81a18eeaa87bd7fa77,2019-09-11T00:00:00.000,Guido Hüni,GLP,Sebastian Vogel,FDP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat
225,Rahmenkredit für Pilotversuche von Asphaltkoll...,Motion,24da9276dc184be6bcdcc19f8ca10dc4,2021-12-08T00:00:00.000,Markus Merki,GLP,Peter Anderegg,EVP,Abgeschlossen,Umwandlung in Postulat,Umwandlung in Postulat


In [20]:
#Um schneller zum Ziel zu kommen, lasse ich mir die urls zu den Geschäften automatisch erstellen
df_umwandlung = df_partei_final[df_partei_final["_entscheidbereinigt"] == "Umwandlung in Postulat"]

In [21]:
url_liste = []
base = "https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid="

for elem in df_umwandlung["ID"]:
    url_liste.append(base + elem)
    

In [22]:
url_liste

['https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=dc9c3c794faf4fc2bbe95377f4a88d98',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=b3b7b404584d4a0b9232f4e9f1a3bd5f',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=ade2db27c44e40c0927562f5aaa2292a',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=b29d97daf827426eb59a265c7c7608b8',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=3e4d684d690549d0b8e4e4f5adbc47b3',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=559cba49e3c24a81a18eeaa87bd7fa77',
 'https://www.gemeinderat-zuerich.ch/geschaefte/detail.php?gid=24da9276dc184be6bcdcc19f8ca10dc4']

In [23]:
#In einem ersten Schritt passe ich nun die umgewandelten Postulate an, die abgelehnt wurden
#Mit dem loc-Befehl kann man über die Index-Nr. die richtige Zeile anwählen und dann die Spalte
df_test = df_partei_final.copy()
df_test.loc[5, "_entscheidbereinigt"] = "Ablehnung"

In [24]:
#Ein Kontrollblick
pd.set_option('display.max_rows', None)
df_test

,Titel,Geschäftsart,ID,Datum,Erstunterzeichner,Partei Erstunterzeichner,Mitunterzeichner,Partei Mitunterzeichner,Geschäftsstatus,Entscheid,_entscheidbereinigt
0,"Juliastrasse, Einrichtung einer Begegnungszone...",Postulat,1ae08ec9dfc0465ba1c5491de2584218,2014-02-05T00:00:00.000,Jean-Daniel Strub,SP,Martin Luchsinger,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
1,Kommunale Wohnüberbauung auf dem Tramdepot Har...,Postulat,a5573cb80d054660a8565a27eab1a841,2014-02-26T00:00:00.000,Martin Luchsinger,GLP,Jean-Claude Virchaux,CVP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
2,Durchgehende Veloverbindung aus dem Lettenquar...,Postulat,94f2be0b2acd4eab8e6d7c743304b799,2014-03-05T00:00:00.000,Simone Brander,SP,Guido Trevisan,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
3,Studie zur Wirkung der Mobilitätskampagne des ...,Schriftliche Anfrage,ef16480862ec4387991f9b736a9378ba,2014-03-12T00:00:00.000,Samuel Dubno,GLP,Walter Angst,AL,Abgeschlossen,NaN,NaN
4,Einführung einer Jugendinitiative als Instrume...,Postulat,4940b23f724d4d6cb4a21aed8f043f63,2014-03-19T00:00:00.000,Isabel Garcia,FDP,Matthias Wiesmann,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
5,"Anpassung der Datenschutzverordnung, Streichun...",Motion,dc9c3c794faf4fc2bbe95377f4a88d98,2014-04-02T00:00:00.000,Samuel Dubno,GLP,Ann-Catherine Nabholz,GLP,Abgeschlossen,Umwandlung in Postulat,Ablehnung
6,Reduktion der Anzahl Videokameras zur Vandalis...,Postulat,925a24cf6b96447788347ec150ec6acb,2014-04-02T00:00:00.000,Ann-Catherine Nabholz,GLP,Samuel Dubno,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
7,"Holzschlag im Stöckentobel, Hintergründe zur g...",Schriftliche Anfrage,6fd2772bdf514acd9f81828525388f2b,2014-04-02T00:00:00.000,Ann-Catherine Nabholz,GLP,Guido Trevisan,GLP,Abgeschlossen,NaN,NaN
8,Überprüfung und allfällige Anpassung der Recht...,Postulat,c9178751aeda40548c080356c30f3c20,2014-04-09T00:00:00.000,Guido Hüni,GLP,Maleica Landolt,GLP,Abgeschlossen,"Überweisung, Frist 24 Monate",Überweisung
9,"Teilrevision der Bau- und Zonenordnung (BZO), ...",Schriftliche Anfrage,f8f77939940040cca65ba1859cdddbc1,2014-04-16T00:00:00.000,Guido Trevisan,GLP,Gian von Planta,GLP,Abgeschlossen,NaN,NaN


In [25]:
pd.reset_option('display.max_rows')

In [26]:
#Nun kann ich alle übrigen umgewandelten Postulate zu "Überweisung" ändern
df_test['_entscheidbereinigt'] = df_test['_entscheidbereinigt'].replace('Umwandlung in Postulat', 'Überweisung')

In [27]:
#Diese Resultate speichern
df_test.to_csv('GLP_Vorstoesse_14_23_mit_Resultat_bereinigt.csv', index=False)